# Module 1: Session Setup
## Run This Notebook First Every Time You Start a New Session!

---

This notebook installs all required packages, sets up the HITRAN database, and verifies your environment is ready.

**Why is this needed?**

To save costs on GPU Droplets, we snapshot the boot disk and destroy the Droplet between sessions. While your notebook files and data directories persist in the snapshot, some installed Python packages or environment state may need to be refreshed. This notebook ensures a consistent, working environment every time.

**Time required:** ~3-4 minutes (mostly downloading PyTorch)

---

## Course Configuration

All course-wide parameters are defined here in one place. Every subsequent cell references these variables, so **always run this cell first**.

In [ ]:
from pathlib import Path

# ── Project paths ──────────────────────────────────────────
WORKING_DIR = Path.cwd()
DATA_DIR    = WORKING_DIR / 'data'
HITRAN_DIR  = DATA_DIR / 'hitran'
SPECTRA_DIR = DATA_DIR / 'spectra'
DATASET_DIR = DATA_DIR / 'datasets'
MODEL_DIR   = WORKING_DIR / 'models'
OUTPUT_DIR  = WORKING_DIR / 'outputs'

# ── HITRAN / spectroscopy parameters ───────────────────────
MOLECULE_ID  = 6          # CH4
ISOTOPE_ID   = 1          # Main isotopologue (12CH4)
NU_MIN       = 4383.0     # cm⁻¹  — start of wavenumber range
NU_MAX       = 4386.0     # cm⁻¹  — end of wavenumber range
TABLE_NAME   = 'CH4_4383_4386'

# ── Default simulation environment ────────────────────────
DEFAULT_TEMP     = 296.0  # K   (HITRAN reference temperature)
DEFAULT_PRESSURE = 1.0    # atm
MOLE_FRACTION    = 0.01   # 1% CH4 (for absorption coefficient calc)
WAVENUMBER_STEP  = 0.001  # cm⁻¹  — spectral resolution

# ── PyTorch / ROCm install URL ────────────────────────────
PYTORCH_INDEX_URL = 'https://download.pytorch.org/whl/rocm6.2'

print(f"Working directory : {WORKING_DIR}")
print(f"HITRAN directory  : {HITRAN_DIR}")
print(f"Table name        : {TABLE_NAME}")
print(f"Wavenumber range  : {NU_MIN} – {NU_MAX} cm⁻¹")
print(f"PyTorch index     : {PYTORCH_INDEX_URL}")
print("\n✅ Configuration loaded.")

## Step 1: Install Core Packages

This installs PyTorch with ROCm support for AMD GPUs.

In [ ]:
import subprocess, sys

print("="*60)
print("STEP 1: Installing PyTorch + ROCm")
print("="*60)
print("This takes ~2-3 minutes (downloading ~4 GB)...\n")

subprocess.check_call([
    sys.executable, '-m', 'pip', 'install',
    'torch', 'torchvision',
    '--index-url', PYTORCH_INDEX_URL,
    '--quiet'
])

print("\n✅ PyTorch installed!")

## Step 2: Install Course Dependencies

Additional packages needed for the course.

In [ ]:
import subprocess, sys

print("="*60)
print("STEP 2: Installing Course Dependencies")
print("="*60)

dep_groups = [
    (['hitran-api'],                                    'hitran-api'),
    (['numpy', 'matplotlib', 'seaborn'],                'numpy, matplotlib, seaborn'),
    (['h5py', 'tqdm', 'pyyaml'],                        'h5py, tqdm, pyyaml'),
    (['ultralytics', 'albumentations',
      'opencv-python-headless'],                        'ultralytics, albumentations, opencv'),
]

for pkgs, label in dep_groups:
    subprocess.check_call(
        [sys.executable, '-m', 'pip', 'install'] + pkgs + ['--quiet']
    )
    print(f"✓ {label}")

print("\n✅ All dependencies installed!")

## Step 3: Set Up Project Directories

Create the directory structure for our course data.

In [ ]:
print("="*60)
print("STEP 3: Setting Up Project Directories")
print("="*60)

for d in [DATA_DIR, HITRAN_DIR, SPECTRA_DIR, DATASET_DIR, MODEL_DIR, OUTPUT_DIR]:
    d.mkdir(parents=True, exist_ok=True)
    print(f"✓ {d}")

print("\n✅ Directories created!")

## Step 4: Set Up HITRAN Database

Download CH₄ spectroscopic data for our wavenumber range (4383–4386 cm⁻¹).

In [ ]:
print("="*60)
print("STEP 4: Setting Up HITRAN Database")
print("="*60)

import hapi

hapi.db_begin(str(HITRAN_DIR))
print(f"HITRAN data directory: {HITRAN_DIR}")

print(f"\nFetching CH₄ data from HITRAN...")
print(f"  Molecule: CH₄ (ID={MOLECULE_ID})")
print(f"  Isotope: ¹²CH₄ (ID={ISOTOPE_ID})")
print(f"  Wavenumber range: {NU_MIN} – {NU_MAX} cm⁻¹")

# Fetch (downloads if not cached, skips if files already exist)
try:
    hapi.fetch(
        TableName=TABLE_NAME,
        M=MOLECULE_ID,
        I=ISOTOPE_ID,
        numin=NU_MIN,
        numax=NU_MAX
    )
    print("\n✓ Data fetched from HITRAN server")
except Exception as e:
    print(f"\n✓ Using cached data (or: {e})")

# Verify data
try:
    nu_lines = hapi.getColumn(TABLE_NAME, 'nu')
    sw_lines = hapi.getColumn(TABLE_NAME, 'sw')
    print(f"\nSpectral lines found: {len(nu_lines)}")
    for i, (nu, sw) in enumerate(zip(nu_lines[:5], sw_lines[:5])):
        print(f"  Line {i+1}: ν = {nu:.4f} cm⁻¹, Intensity = {sw:.2e}")
    if len(nu_lines) > 5:
        print(f"  ... and {len(nu_lines)-5} more lines")
    print("\n✅ HITRAN database ready!")
except Exception as e:
    print(f"\n⚠️ Could not verify HITRAN data: {e}")

## Step 5: Verify GPU Access

Confirm PyTorch can see the AMD GPU via ROCm's HIP layer.

In [ ]:
print("="*60)
print("STEP 5: Verifying GPU Access")
print("="*60)

import torch

print(f"\nPyTorch version: {torch.__version__}")
print(f"ROCm available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU count: {torch.cuda.device_count()}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    
    total_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU memory: {total_mem:.0f} GB")
    
    # Quick compute test
    print("\nRunning GPU compute test...")
    x = torch.randn(5000, 5000, device='cuda')
    y = torch.matmul(x, x)
    torch.cuda.synchronize()
    print("✓ Matrix multiplication: PASSED")
    
    # Gradient test
    x = torch.randn(100, requires_grad=True, device='cuda')
    y = (x ** 2).sum()
    y.backward()
    print("✓ Gradient computation: PASSED")
    
    print("\n✅ GPU is working correctly!")
else:
    print("\n❌ GPU not detected! Check ROCm installation.")

## Step 6: Verify All Packages

Final check that all required packages are importable.

In [ ]:
print("="*60)
print("STEP 6: Verifying All Packages")
print("="*60)

packages = {
    'torch': 'torch',
    'torchvision': 'torchvision',
    'numpy': 'numpy',
    'matplotlib': 'matplotlib',
    'seaborn': 'seaborn',
    'hapi': 'hapi',
    'h5py': 'h5py',
    'tqdm': 'tqdm',
    'yaml': 'yaml',
    'cv2': 'cv2',
    'ultralytics': 'ultralytics',
    'albumentations': 'albumentations',
}

all_ok = True
print("\nPackage Status:")
print("-" * 40)

for name, module in packages.items():
    try:
        mod = __import__(module)
        version = getattr(mod, '__version__', 'installed')
        print(f"✓ {name}: {version}")
    except ImportError:
        print(f"✗ {name}: NOT FOUND")
        all_ok = False

if all_ok:
    print("\n✅ All packages verified!")
else:
    print("\n⚠️ Some packages missing. Re-run Step 2.")

---
## ✅ Session Setup Complete!

Run the cell below to see a summary and confirm everything is ready.

In [ ]:
print("\n" + "="*60)
print("           SESSION SETUP SUMMARY")
print("="*60)

import torch

# Check GPU
gpu_ok = torch.cuda.is_available()
gpu_status = f"✅ {torch.cuda.get_device_name(0)}" if gpu_ok else "❌ Not detected"

# Check HITRAN
hitran_header = HITRAN_DIR / f"{TABLE_NAME}.header"
hitran_ok = hitran_header.exists()
hitran_status = "✅ Ready" if hitran_ok else "❌ Not found"

# Check directories
dirs_ok = all(d.exists() for d in [DATA_DIR, HITRAN_DIR, MODEL_DIR])
dirs_status = "✅ Created" if dirs_ok else "❌ Missing"

print(f"""
┌─────────────────────────────────────────────────────────┐
│  Component          │  Status                          │
├─────────────────────────────────────────────────────────┤
│  PyTorch + ROCm     │  {torch.__version__:<30} │
│  GPU                │  {gpu_status:<30} │
│  HITRAN Database    │  {hitran_status:<30} │
│  Directories        │  {dirs_status:<30} │
└─────────────────────────────────────────────────────────┘
""")

if gpu_ok and hitran_ok and dirs_ok:
    print("🎉 ALL SYSTEMS GO! You're ready to proceed to Module 2.")
    print("\nNext steps:")
    print("  1. Open Module_02_Physics_Background.ipynb")
    print("  2. Or open Module_03_HITRAN_Simulation.ipynb")
else:
    print("⚠️  Some issues detected. Please review the steps above.")

---

## Quick Reference: HITRAN Setup for Other Notebooks

After running this Session Setup, other notebooks can use HITRAN like this:

```python
import hapi
from pathlib import Path

# Point to our cached HITRAN data
HITRAN_DIR = Path('./data/hitran')
hapi.db_begin(str(HITRAN_DIR))

# Use the pre-fetched CH4 data
TABLE_NAME = 'CH4_4383_4386'

# Generate spectrum
nu, coef = hapi.absorptionCoefficient_Voigt(
    SourceTables=TABLE_NAME,
    Components=[(6, 1, 0.01)],  # (molecule_id, isotope_id, mole_fraction)
    Environment={'T': 296, 'p': 1.0},
    WavenumberRange=[4383, 4386],
    WavenumberStep=0.001
)
```

---

**Module 1 Complete!** Proceed to Module 2 for physics background, or Module 3 to start simulating spectra.